Creating the MCP Server


fetch_wikipedia_info tool : searches Wikipedia for a given topic and returns structured information, including the article’s title, a summary, and a URL to the best-matching page.

In [ ]:
import wikipedia
from mcp.server.fastmcp import FastMCP

In [ ]:
mcp = FastMCP("WikipediaSearch")

In [ ]:
@mcp.tool()
def fetch_wikipedia_info(query: str) -> dict:
    """
    Search Wikipedia for a topic and return title, summary, and URL of the best match.
    """
    try:
        search_results = wikipedia.search(query)
        if not search_results:
            return {"error": "No results found for your query."}

        best_match = search_results[0]
        page = wikipedia.page(best_match)

        return {
            "title": page.title,
            "summary": page.summary,
            "url": page.url
        }

    except wikipedia.DisambiguationError as e:
        return {
            "error": f"Ambiguous topic. Try one of these: {', '.join(e.options[:5])}"
        }

    except wikipedia.PageError:
        return {
            "error": "No Wikipedia page could be loaded for this query."
        }


In [ ]:
# Run the MCP server
if __name__ == "__main__":
    print("Starting MCP Wikipedia Server...")

list_wikipedia_sections tool : It takes a topic as input and returns a list of section titles from the corresponding Wikipedia page. This allows the AI assistant to guide users through the content or offer them options to dive deeper into specific sections.

In [ ]:
@mcp.tool()
def list_wikipedia_sections(topic: str) -> dict:
    """
    Return a list of section titles from the Wikipedia page of a given topic.
    """
    try:
        page = wikipedia.page(topic)
        sections = page.sections
        return {"sections": sections}
    except Exception as e:
        return {"error": str(e)}

get_section_content tool : which will fetch the full text of a specific section from a Wikipedia article. This is useful when a user asks to “show me the early life section” or “give me the research contributions part” of an article.

In [ ]:
@mcp.tool()
def get_section_content(topic: str, section_title: str) -> dict:
    """
    Return the content of a specific section in a Wikipedia article.
    """
    try:
        page = wikipedia.page(topic)
        content = page.section(section_title)
        if content:
            return {"content": content}
        else:
            return {"error": f"Section '{section_title}' not found in article '{topic}'."}
    except Exception as e:
        return {"error": str(e)}